In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense
from sklearn.model_selection import train_test_split
import sklearn
from keras import optimizers
adam = optimizers.adam(lr=0.001)
from sklearn.preprocessing import MinMaxScaler
import os
import glob
sc = MinMaxScaler(feature_range=(0,1))

Using TensorFlow backend.


In [10]:
path = r'c:\Users\user\Desktop\diploma\python_scripts\\'
extension = 'csv'
os.chdir(path)
files = glob.glob('*.{}'.format(extension))

In [11]:
def predict_price(filename, train_test_split, epochs, validation_split):
    data = pd.read_csv(filename, index_col = "Date")
    data_train = data.iloc[:round(int(len(data)*train_test_split)), :]
    data_test = data.iloc[round(int(len(data)*train_test_split)):,:]
    X_train = data_train.iloc[:, 3:4]
    X_train_scaled = sc.fit_transform(X_train)
    x_train = []
    y_train = []
    for i in range(60, X_train.shape[0]):
        x_train.append(X_train_scaled[i-60:i, 0])
        y_train.append(X_train_scaled[i, 0])
    x_train_array = np.array(x_train)
    y_train_array = np.array(y_train)
    x_train_input = np.reshape(x_train_array, (x_train_array.shape[0], x_train_array.shape[1], 1))
    model = Sequential()
    model.add(LSTM(units=150,return_sequences=True,input_shape=(x_train_input.shape[1], 1)))
    model.add(Dropout(0.2))
    model.add(LSTM(units=100,return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units=100,return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50))
    model.add(Dropout(0.2))
    model.add(Dense(units=1))
    model.compile(optimizer='adam',loss='mean_squared_error')
    model.fit(x_train_input,y_train_array, epochs=epochs,batch_size=64, validation_split = validation_split, verbose = 1)
    real_close_price = data_test.iloc[:, 3:4]
    all_data = pd.concat((data_train['Close'], data_test['Close']), axis = 0)
    inputs = all_data[len(all_data) - len(data_test) - 60:].values
    inputs = inputs.reshape(-1,1)
    inputs = sc.transform(inputs)
    X_test = []
    for i in range(60, inputs.shape[0]):
        X_test.append(inputs[i-60:i, 0])
    X_test_array = np.array(X_test)
    X_test_input = np.reshape(X_test_array, (X_test_array.shape[0], X_test_array.shape[1], 1))
    predicted_close_price_tf = model.predict(X_test_input)
    predicted_close_price = sc.inverse_transform(predicted_close_price_tf)
    return predicted_close_price


In [12]:
def calc_da(filename, train_test_split, predicted_data):
    data = pd.read_csv(filename, index_col = "Date")
    data_test = data.iloc[round(int(len(data)*train_test_split)):,:]
    real_close_price = np.array(data_test.iloc[:, 3:4])
    count = 0
    for i in range(len(real_close_price)-1):
        if (real_close_price[i+1]-real_close_price[i])*(predicted_data[i+1]-predicted_data[i])>0:
            count += 1
    DA = count * 100 / (len(real_close_price)-1)
    return DA 

In [13]:
def strategy(filename, train_test_split, predicted_data):
    data = pd.read_csv(filename, index_col = "Date")
    data_test = data.iloc[round(int(len(data)*train_test_split)):,:]
    real_open_price = np.array(data_test.iloc[:, 0:1])
    real_close_price = np.array(data_test.iloc[:, 3:4])
    pred_reward = 0
    pred_reward_lst = []
    for i in range(len(real_open_price)-1):
        if predicted_data[i] - real_open_price[i] > 0:
            pred_reward += (real_close_price[i] - real_open_price[i])
            pred_reward_lst.append(real_close_price[i] - real_open_price[i])
        else:
            pred_reward += (real_open_price[i] - real_close_price[i])
            pred_reward_lst.append(real_close_price[i] - real_open_price[i])
    return pred_reward

In [9]:
calc_da(files[1], 0.8, predict_price(files[1], 0.8, 2))

Train on 4104 samples, validate on 457 samples
Epoch 1/2
4104/4104 [==============================] - 51s 12ms/step - loss: 0.0039 - val_loss: 0.0023
Epoch 2/2
4104/4104 [==============================] - 43s 10ms/step - loss: 5.4490e-04 - val_loss: 0.0019


52.72727272727273

In [16]:
calc_da(files[2], 0.8, predict_price(files[2], 0.8, 15))

Train on 8645 samples, validate on 961 samples
Epoch 1/15
8645/8645 [==============================] - 162s 19ms/step - loss: 0.0040 - val_loss: 0.0044
Epoch 2/15
8645/8645 [==============================] - 149s 17ms/step - loss: 0.0014 - val_loss: 0.0032
Epoch 3/15
8645/8645 [==============================] - 152s 18ms/step - loss: 0.0012 - val_loss: 0.0064
Epoch 4/15
8645/8645 [==============================] - 158s 18ms/step - loss: 0.0014 - val_loss: 0.0018
Epoch 5/15
8645/8645 [==============================] - 159s 18ms/step - loss: 9.9455e-04 - val_loss: 0.0021
Epoch 6/15
8645/8645 [==============================] - 164s 19ms/step - loss: 9.1810e-04 - val_loss: 0.0015
Epoch 7/15
8645/8645 [==============================] - 170s 20ms/step - loss: 9.2014e-04 - val_loss: 0.0020
Epoch 8/15
8645/8645 [==============================] - 171s 20ms/step - loss: 8.3507e-04 - val_loss: 0.0038
Epoch 9/15
8645/8645 [==============================] - 178s 21ms/step - loss: 8.5658e-04 - val_l

49.25496688741722

In [7]:
calc_da(files[5], 0.8, predict_price(files[5], 0.8, 5))

Train on 5423 samples, validate on 603 samples
Epoch 1/5
5423/5423 [==============================] - 62s 11ms/step - loss: 0.0039 - val_loss: 3.3391e-04
Epoch 2/5
5423/5423 [==============================] - 55s 10ms/step - loss: 0.0011 - val_loss: 1.3547e-04
Epoch 3/5
5423/5423 [==============================] - 57s 10ms/step - loss: 0.0010 - val_loss: 1.3554e-04
Epoch 4/5
5423/5423 [==============================] - 58s 11ms/step - loss: 9.2260e-04 - val_loss: 1.5507e-04
Epoch 5/5
5423/5423 [==============================] - 59s 11ms/step - loss: 8.7704e-04 - val_loss: 1.1963e-04


48.19197896120973

In [9]:
calc_da(files[5], 0.8, predict_price(files[5], 0.8, 1))

Train on 5423 samples, validate on 603 samples
Epoch 1/1
5423/5423 [==============================] - 75s 14ms/step - loss: 0.0038 - val_loss: 6.7268e-04


49.2439184746877

In [16]:
calc_da(files[5], 0.8, predict_price(files[5], 0.7, 2, 0.2))

Train on 4212 samples, validate on 1053 samples
Epoch 1/2
4212/4212 [==============================] - 42s 10ms/step - loss: 0.0046 - val_loss: 3.4747e-04
Epoch 2/2
4212/4212 [==============================] - 38s 9ms/step - loss: 0.0013 - val_loss: 2.8585e-04


49.83563445101907

In [ ]:
pred_price_list = []
for i in range(1, len(files)):
    pred_price_list.append(predict_price(files[i], 0.8, 1))
    #all_reward += strategy(files[i], 0.8, predict_price(files[i], 0.8))
  

Train on 4104 samples, validate on 457 samples
Epoch 1/1
4104/4104 [==============================] - 54s 13ms/step - loss: 0.0025 - val_loss: 0.0022
Train on 8645 samples, validate on 961 samples
Epoch 1/1
8645/8645 [==============================] - 110s 13ms/step - loss: 0.0042 - val_loss: 0.0177
Train on 10516 samples, validate on 1169 samples
Epoch 1/1
10516/10516 [==============================] - 136s 13ms/step - loss: 0.0018 - val_loss: 0.0045
Train on 10516 samples, validate on 1169 samples
Epoch 1/1
10516/10516 [==============================] - 142s 14ms/step - loss: 5.4469e-04 - val_loss: 0.0016
Train on 5423 samples, validate on 603 samples
Epoch 1/1
5423/5423 [==============================] - 82s 15ms/step - loss: 0.0040 - val_loss: 1.4789e-04
Train on 10516 samples, validate on 1169 samples
Epoch 1/1
10516/10516 [==============================] - 157s 15ms/step - loss: 9.8341e-04 - val_loss: 0.0014
Train on 10516 samples, validate on 1169 samples
Epoch 1/1
10516/10516 [

In [7]:
reward_list = []
all_reward = 0
for i in range(1, 10):
    reward_list.append(strategy(files[i], 0.8, predict_price(files[i], 0.8, 1, 0.2)))
    #all_reward += strategy(files[i], 0.8, predict_price(files[i], 0.8))
  
        

Train on 3648 samples, validate on 913 samples
Epoch 1/1
3648/3648 [==============================] - 21s 6ms/step - loss: 0.0016 - val_loss: 0.0016
Train on 7684 samples, validate on 1922 samples
Epoch 1/1
7684/7684 [==============================] - 45s 6ms/step - loss: 0.0021 - val_loss: 0.0018
Train on 9348 samples, validate on 2337 samples
Epoch 1/1
9348/9348 [==============================] - 60s 6ms/step - loss: 0.0013 - val_loss: 0.0014
Train on 9348 samples, validate on 2337 samples
Epoch 1/1
9348/9348 [==============================] - 62s 7ms/step - loss: 2.9568e-04 - val_loss: 0.0014
Train on 4820 samples, validate on 1206 samples
Epoch 1/1
4820/4820 [==============================] - 33s 7ms/step - loss: 0.0049 - val_loss: 4.1569e-04
Train on 9348 samples, validate on 2337 samples
Epoch 1/1
9348/9348 [==============================] - 65s 7ms/step - loss: 5.6996e-04 - val_loss: 0.0035
Train on 9348 samples, validate on 2337 samples
Epoch 1/1
9348/9348 [====================

In [8]:
print(reward_list)

[array([238.080483]), array([-7.240178]), array([-18.729774]), array([59.989854]), array([13.649945]), array([-5.59994]), array([41.480065]), array([-8.859997]), array([47.220243])]


In [9]:
reward = 0
for i in range(len(reward_list)):
    reward += reward_list[i]
print(reward)

[359.990701]


In [6]:
reward_list = []
all_reward = 0
for i in range(10, 20):
    reward_list.append(strategy(files[i], 0.8, predict_price(files[i], 0.8, 1, 0.2)))
    #all_reward += strategy(files[i], 0.8, predict_price(files[i], 0.8))
  

Train on 6181 samples, validate on 1546 samples
Epoch 1/1
6181/6181 [==============================] - 35s 6ms/step - loss: 0.0064 - val_loss: 6.4162e-04
Train on 9348 samples, validate on 2337 samples
Epoch 1/1
9348/9348 [==============================] - 55s 6ms/step - loss: 0.0011 - val_loss: 0.0030
Train on 6426 samples, validate on 1607 samples
Epoch 1/1
6426/6426 [==============================] - 39s 6ms/step - loss: 0.0028 - val_loss: 0.0010
Train on 9348 samples, validate on 2337 samples
Epoch 1/1
9348/9348 [==============================] - 57s 6ms/step - loss: 6.8269e-04 - val_loss: 0.0026
Train on 6426 samples, validate on 1607 samples
Epoch 1/1
6426/6426 [==============================] - 41s 6ms/step - loss: 0.0073 - val_loss: 0.0035
Train on 9348 samples, validate on 2337 samples
Epoch 1/1
9348/9348 [==============================] - 59s 6ms/step - loss: 0.0021 - val_loss: 0.0103
Train on 8621 samples, validate on 2156 samples
Epoch 1/1
8621/8621 [=======================

In [7]:
print(reward_list)

[array([-113.419701]), array([-60.28021]), array([52.950024]), array([-21.119909]), array([11.590065]), array([22.319972]), array([-74.83997]), array([-64.990146]), array([17.880082]), array([-26.0399])]


In [8]:
reward = 0
for i in range(len(reward_list)):
    reward += reward_list[i]
print(reward)

[-255.949693]


In [ ]:
accuracies = []
for i in range(1,11):
    accuracies.append(calc_da(files[i], 0.8, predict_price(files[i], 0.7, 2, 0.2)))
    

Train on 3186 samples, validate on 797 samples
Epoch 1/2
3186/3186 [==============================] - 27s 8ms/step - loss: 0.0032 - val_loss: 0.0025
Epoch 2/2
3186/3186 [==============================] - 24s 8ms/step - loss: 7.9500e-04 - val_loss: 0.0021
Train on 6718 samples, validate on 1680 samples
Epoch 1/2
6718/6718 [==============================] - 60s 9ms/step - loss: 0.0015 - val_loss: 0.0062
Epoch 2/2
6718/6718 [==============================] - 55s 8ms/step - loss: 4.0766e-04 - val_loss: 0.0025
Train on 8173 samples, validate on 2044 samples
Epoch 1/2
8173/8173 [==============================] - 71s 9ms/step - loss: 8.1284e-04 - val_loss: 0.0066
Epoch 2/2
8173/8173 [==============================] - 69s 8ms/step - loss: 2.5301e-04 - val_loss: 0.0063
Train on 8173 samples, validate on 2044 samples
Epoch 1/2
8173/8173 [==============================] - 72s 9ms/step - loss: 9.7009e-04 - val_loss: 0.0038
Epoch 2/2
8173/8173 [==============================] - 71s 9ms/step - loss:

In [1]:
print(accuracies)

NameError: name 'accuracies' is not defined

In [ ]:
calc_da("AMZN.csv", 0.8, funkcia('AMZN.csv', 0.8))

In [ ]:
overnight = 0
for i in range(len(real_open_price)-1):
    if predicted_close_price[i+1] - predicted_close_price[i] > 0:
        overnight += (real_close_price[i+1] - real_close_price[i])
        #real_reward_lst.append(real_close_price[i] - real_open_price[i])
    #else:
     #   overnight += (real_close_price[i] - real_close_price[i+1])
        #real_reward_lst.append(real_close_price[i] - real_open_price[i])

In [ ]:
overnight